In [2]:
import torch
import pytorch_lightning as pl
import torchvision
import matplotlib.pyplot as plt


import sys
sys.path.append('task_1/')
import models
import trainer

In [38]:
import importlib
importlib.reload(models)
importlib.reload(trainer)

<module 'trainer' from 'c:\\Users\\Nirmal\\Documents\\Nirmal_Git\\bitbucket\\aditya_dlrg_fork\\DLRG\\nirmal\\assignment_3\\task_1\\trainer.py'>

In [39]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# initialize models
my_models = trainer.MyModel()

# initialize trainer
my_trainer = pl.Trainer(max_epochs=1, gpus=0)

# train models
my_trainer.fit(my_models, train_loader, test_loader)

#my_models.forward(torch.randn(32, 1, 28, 28))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type      | Params
-----------------------------------------
0 | big_cnn    | BigCNN    | 190 K 
1 | little_cnn | LittleCNN | 1.6 M 
2 | mlp        | MLP       | 535 K 
-----------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.424     Total estimated model params size (MB)


Epoch 0:   8%|▊         | 169/2188 [00:31<06:20,  5.31it/s, loss=0.665, v_num=15]

In [18]:
'''
In ‘main.py’ initialize the dataset, dataloaders, logger, models, and trainer class. Use
argparse to execute the file. In-line commands must include the logging directory’s path,
batch size, number of workers, fast dev run, and number of epochs. Save model weights
in the logging directory as well.
'''

import argparse
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
import torchvision
import torch
import sys
sys.path.append('task_1/')
import models
from trainer import MyModel

def main(args):
    # initialize dataset
    train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
    test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

    # initialize dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=args.num_workers)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, num_workers=args.num_workers)

    # initialize logger
    logger = TensorBoardLogger('logs', name='task_1')

    # initialize models
    big_cnn = models.BigCNN()
    little_cnn = models.LittleCNN()
    mlp = models.MLP()

    # initialize trainer
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=args.max_epochs,
        fast_dev_run=args.fast_dev_run,
        gpus=1 if torch.cuda.is_available() else 0,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=3),
            ModelCheckpoint(monitor='val_loss')
        ]
    )

    # train model
    model = MyModel()
    trainer.fit(model, train_dataloader, test_dataloader)

    # save model weights
    trainer.save_checkpoint('task_1.ckpt')


In [19]:
main()

TypeError: main() missing 1 required positional argument: 'args'